# Model Trainer Module

In [1]:
import os
%pwd

'c:\\Users\\amman\\Documents\\Generative AI\\End-to-End-Text-Summariser-Project\\notebooks'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\amman\\Documents\\Generative AI\\End-to-End-Text-Summariser-Project'

In [18]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass
class ModelTrainerConfig:
    root_dir: Path 
    data_path: Path 
    model_ckpt: Path 
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int
    fp16: bool
    report_to: str

In [4]:
from src.textSummariser.constants import * 
from src.textSummariser.utils.common import create_directories, read_yaml

In [19]:
class ConfigurationManager():
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            eval_strategy=params.eval_strategy,
            eval_steps=params.eval_steps,
            save_steps=int(params.save_steps),
            gradient_accumulation_steps=params.gradient_accumulation_steps,
            fp16=params.fp16,
            report_to=params.report_to,

        )

        return model_trainer_config

In [6]:
from transformers import (AutoTokenizer,
                        AutoModelForSeq2SeqLM,
                        TrainingArguments,
                        Trainer,
                        DataCollatorForSeq2Seq)
import torch 
from datasets import load_from_disk

c:\Users\amman\Documents\Generative AI\End-to-End-Text-Summariser-Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-02-17 22:56:10,122: INFO: config: PyTorch version 2.6.0+cu126 available.]


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [20]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config 

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # Load the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps, per_device_train_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            eval_strategy=self.config.eval_strategy, eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps, gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            fp16=self.config.fp16, report_to=self.config.report_to
        )

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                          data_collator=data_collator, tokenizer=tokenizer,
                          train_dataset=dataset_samsum_pt['test'],
                          eval_dataset=dataset_samsum_pt['validation'])
        
        trainer.train()

        # Save the model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        # Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [ ]:
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

Found existing installation: transformers 4.49.0
Uninstalling transformers-4.49.0:
  Successfully uninstalled transformers-4.49.0
Found existing installation: accelerate 1.4.0
Uninstalling accelerate-1.4.0:
  Successfully uninstalled accelerate-1.4.0
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached accelerate-1.4.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
Using cached accelerate-1.4.0-py3-none-any.whl (342 kB)


In [ ]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-02-17 23:01:00,078: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2025-02-17 23:01:00,082: INFO: common: yaml file: params.yaml loaded successfully.]
[2025-02-17 23:01:00,084: INFO: common: Created directory at: artifacts]
[2025-02-17 23:01:00,086: INFO: common: Created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\amman\AppData\Local\Temp\ipykernel_8420\129166262.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,


Step,Training Loss,Validation Loss
